# Version 4 

L'objectif de cette version est d'utiliser les données historiques et le nouveau cadre de simulation fait en version 3 pour pouvoir implémenter :
1. Un algo ultra basique comme fait en version 2
2. Un algo un peu plus élaboré basé sur le principe du filtre de Kalman

Questionnement :
1. Pas d'interet de faire tourner l'ancien algo car il va forcément bien marcher même sur le nouveau cadre étant donnée que le nombre de seconde dans une journée est connue (horaires fixes d'ouverture des br).
2. Si jamais je devais le faire tourner, ça serait sur les données historiques générées ou faire l'implémentation "en temps réel" en parallèle de la simulation ? 